In [4]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
%matplotlib inline

**Load the Dataset**

In [0]:
df_inp=pd.read_csv('/content/drive/My Drive/CNN/dog-breed-identification/corpus/blogtext.csv')

In [0]:
df_inp=df_inp.head(5000)

Text Preprocess

In [0]:
df_inp['text']=df_inp['text'].astype('str')
df_inp['age']=df_inp['age'].astype('str')
#Convert to lower case
df_inp['text']=df_inp['text'].apply(lambda x: x.lower())

In [9]:
df_text=pd.DataFrame()
df_text['text']=df_inp['text']
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 1 columns):
text    5000 non-null object
dtypes: object(1)
memory usage: 39.2+ KB


In [0]:
#Remove the unwanted characters
import re
df_text=df_text.applymap(lambda s:re.sub("[^0-9a-z #+_]", " ",s))

In [0]:
#Remove the unwanted space
df_text=df_text.applymap(lambda s:s.strip())

Merge the columns

In [0]:
df_text['labels']=df_inp[['gender','age','topic','sign']].values.tolist()
#df_text['labels']=df_text['labels'].values.tolist()

In [34]:
df_text.head()

,text,labels
0,info has been found + 100 pages and 4 5 mb...,"[male, 15, Student, Leo]"
1,these are the team members drewes van der l...,"[male, 15, Student, Leo]"
2,in het kader van kernfusie op aarde maak je ...,"[male, 15, Student, Leo]"
3,testing testing,"[male, 15, Student, Leo]"
4,thanks to yahoo s toolbar i can now capture ...,"[male, 33, InvestmentBanking, Aquarius]"


In [0]:
from sklearn.model_selection import train_test_split

**Train & Test Split **

In [0]:

X = df_text['text'] #independent variable
y = df_text['labels'] #target

# split the new DataFrame into training and testing sets [Default test size = 25%]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

**Vectorize the features**

In [36]:

#Remove stop words
vect = CountVectorizer(ngram_range=(1, 2),stop_words='english')

X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
X_train_dtm.shape

(3750, 239494)

In [37]:
X_test_dtm.shape

(1250, 239494)

**Dictionary to get the count of every label**

In [17]:
print(df_inp['gender'].value_counts().to_dict())
print(df_inp['age'].value_counts().to_dict())
print(df_inp['sign'].value_counts().to_dict())
print(df_inp['topic'].value_counts().to_dict())

{'male': 3294, 'female': 1706}
{'35': 2307, '34': 540, '24': 353, '15': 339, '17': 331, '25': 268, '14': 170, '23': 137, '33': 101, '26': 96, '27': 86, '39': 79, '16': 67, '36': 60, '37': 19, '45': 14, '41': 14, '42': 9, '46': 7, '44': 3}
{'Aries': 2483, 'Sagittarius': 704, 'Libra': 414, 'Scorpio': 408, 'Aquarius': 329, 'Leo': 190, 'Taurus': 100, 'Cancer': 94, 'Gemini': 86, 'Capricorn': 84, 'Pisces': 67, 'Virgo': 41}
{'Technology': 2332, 'indUnk': 1381, 'Student': 569, 'Engineering': 119, 'Education': 118, 'BusinessServices': 87, 'Sports-Recreation': 75, 'InvestmentBanking': 70, 'Communications-Media': 61, 'Non-Profit': 47, 'Science': 33, 'Arts': 31, 'Internet': 20, 'Consulting': 16, 'Banking': 16, 'Automotive': 14, 'Religion': 4, 'Law': 3, 'Accounting': 2, 'Museums-Libraries': 2}


Binarizer

In [0]:

from sklearn.preprocessing import MultiLabelBinarizer
#classes=np.unique(y_train)
binarizer = MultiLabelBinarizer()
#test=['africa','india','america']
test=pd.DataFrame()
test=[['male','44','single','leao'],['female','34','single','leo']]
df_bin_train=binarizer.fit_transform(y_train)
df_bin_test=binarizer.transform(y_test)

In [52]:
np.unique(y_train).shape

(95,)

In [39]:
 binarizer.inverse_transform(df_bin_train)
 #df_bin_train

[('35', 'Aries', 'Technology', 'male'),
 ('24', 'Engineering', 'Libra', 'male'),
 ('24', 'Engineering', 'Libra', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('17', 'Aquarius', 'indUnk', 'male'),
 ('23', 'Sagittarius', 'indUnk', 'male'),
 ('15', 'Libra', 'Student', 'female'),
 ('34', 'Sagittarius', 'female', 'indUnk'),
 ('15', 'Aquarius', 'Student', 'male'),
 ('34', 'Sagittarius', 'female', 'indUnk'),
 ('35', 'Aries', 'Technology', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('34', 'Sagittarius', 'female', 'indUnk'),
 ('25', 'BusinessServices', 'Sagittarius', 'male'),
 ('23', 'Sagittarius', 'indUnk', 'male'),
 ('34', 'Sagittarius', 'female', 'indUnk'),
 ('35', 'Aries', 'Technology', 'male'),
 ('23', 'Sagittarius', 'Student', 'male'),
 ('23', 'Sagittarius', 'indUnk', 'male'),
 ('27', 'Aquarius', 'Education', 'female'),
 ('15', 'Libra', 'Science', 'male'),
 ('34', 'Sagittarius', 'female', 'indUnk'),
 ('35', 'Aries', 'Technology', 'male')

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

**OnevsRest Classifier**

In [41]:
clf=LogisticRegression(solver='lbfgs')
#clf=OneVsRestClassifier(MultinomialNB())
clf=OneVsRestClassifier(clf)
clf.fit(X_train_dtm,df_bin_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [0]:
pred=clf.predict(X_test_dtm)

In [0]:
pred[0]
df_bin_test[0]

array([1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 0])

**F1 Score  , Accuracy, Precision & Recall**

In [43]:
#precision, recall, fscore, support = score(df_bin_train, pred)
from sklearn.metrics import accuracy_score


print(accuracy_score(df_bin_test, pred))

   # print(recall_score(y_test, y_pred, average="macro"))  

0.5408


In [44]:
from sklearn.metrics import recall_score
recall_score(df_bin_test, pred,average='weighted')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.6724

In [45]:
from sklearn.metrics import precision_score
precision_score(df_bin_test, pred,average='weighted')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.7930529324851555

In [56]:
from sklearn.metrics import f1_score
f1_score(df_bin_test, pred, average='weighted')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.6951628270325698

In [47]:
df_bin_test.shape

(1250, 54)

In [48]:
pred.shape

(1250, 54)

In [49]:
X_test[:5]

2764    why on earth would you do that  and if you mus...
4767    hello    i have an audition tomorrow    i m qu...
3814    holy crap  more then your mom  they must suck ...
3499          what do you all think of adbusters magazine
2735                correction       annoyed  r epublican
Name: text, dtype: object

In [50]:
y_test[:5]

2764    [male, 35, Technology, Aries]
4767     [female, 16, Student, Libra]
3814      [male, 14, indUnk, Scorpio]
3499    [male, 35, Technology, Aries]
2735    [male, 35, Technology, Aries]
Name: labels, dtype: object

**Print 5 labels for Precition & Actual **

In [51]:
pred[:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1]])

In [53]:
binarizer.inverse_transform(pred[:5])

[('35', 'Aries', 'Technology', 'male'),
 ('female',),
 ('Aries', 'Technology', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('35', 'Aries', 'Technology', 'male')]

In [54]:
binarizer.inverse_transform(df_bin_train[:5])

[('35', 'Aries', 'Technology', 'male'),
 ('24', 'Engineering', 'Libra', 'male'),
 ('24', 'Engineering', 'Libra', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('17', 'Aquarius', 'indUnk', 'male')]